# Examine model
We want to examine how the mikenet model works by checking:
1. Whether Chang's weights produce HS04 fig. 12 result
2. What are the Input **direction** and **magnitude**

In [ ]:
import os
import tensorflow as tf
import numpy as np
import meta, modeling, evaluate
os.chdir(os.environ.get("TF_ROOT"))

## Restore to Chang's weights

In [ ]:
cfg = meta.Config.from_json('models/surgery/model_config.json')
model = modeling.MyModel(cfg)
model.build()
ckpt = tf.train.Checkpoint(model=model)
ckpt.restore(os.path.join(cfg.checkpoint_folder,'epoch-1'))
[print(f'{w.name} mean: {w.numpy().mean()}') for w in model.weights]

## Test with mn_r100 testset

In [ ]:
mn_r100 = evaluate.load_testset('mn_r100')
model.set_active_task('triangle')
y_pred = model([mn_r100['ort']] * 12)

![HS04 figure 12](/triangle_model/references/hs04_fig12.png)

In [ ]:
mn_r100['']

## Checking the sparse representation still looks good

In [ ]:
def dense_to_sparse(dense: np.array) -> list:
    """Convert dense representation to sparse representation."""
    sparse = []
    for i, unit in enumerate(dense):
        if unit == 1:
            sparse.append(i)
    return sparse

def word_to_sparse(testset: dict, word: str) -> dict:
    """Convert word to sparse representation."""
    word_idx = testset['item'].index(word)
    return {f"{x}: {dense_to_sparse(mn_r100[x][word_idx])}" for x in ['ort', 'pho', 'sem']}


word_to_sparse(mn_r100, 'close')

In [ ]:
# TF style naming to HS04 naming for reference

name_map = {
    'input_hos_hs': 'OS',
    'input_hop_hp': 'OP',
    'input_hps_hs': 'PS',
    'input_css_cs': 'CS'
}

In [ ]:
def expand_over_time(x, n_times):
    """Expand representation to n_times to axis 0"""
    x = tf.Variable(x, dtype=tf.float32)
    x = tf.expand_dims(x, axis=0)
    x = tf.tile(x, [n_times, 1, 1])
    return x

sem = expand_over_time(mn_r100['sem'], 13)
pho = expand_over_time(mn_r100['pho'], 13)

# Checking
[tf.assert_equal(sem[i], tf.cast(mn_r100['sem'], dtype=tf.float32)) for i in range(13)]
[tf.assert_equal(pho[i], tf.cast(mn_r100['pho'], dtype=tf.float32)) for i in range(13)]

In [ ]:
def get_inputs(y: tf.Tensor, mask: tf.Tensor) -> list:
    """Get the input over time tick.
    Assumed dimensions equal between y and mask:
        y: (timetick, word, unit)
    """
    assert y.shape == mask.shape
    masked_y = mask * y
    mean_y = tf.reduce_sum(masked_y, axis=2) / tf.reduce_sum(mask, axis=2)  # Average over unit dimension among on nodes
    return tf.reduce_mean(mean_y, axis=1).numpy().tolist()

In [ ]:
from metrics import CosineSemanticAccuracy
acc = CosineSemanticAccuracy()
acc(y_true=tf.cast(mn_r100['sem'], tf.float32), y_pred=y_pred['sem'][-2])

In [ ]:
# Raw input before TAI
op = get_inputs(y_pred['input_hop_hp'], mask=pho)
os = get_inputs(y_pred['input_hos_hs'], mask=sem)
ps = get_inputs(y_pred['input_hps_hs'], mask=sem)
cs = get_inputs(y_pred['input_css_cs'], mask=sem)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(op, label='O->P')
plt.plot(os, label='O->S')
plt.plot(ps, label='P->S')
plt.plot(cs, label='C->S')
plt.legend(loc="lower right")
plt.show()

# Double check with troubleshooting module

In [ ]:
from troubleshooting import Diagnosis

d = Diagnosis("surgery")
d.eval('mn_r100', task="triangle", epoch=1)
    


In [ ]:
class MNDiagnosis(Diagnosis):

# From old codes

In [ ]:
import pandas as pd

layer = 'sem'
target_word = 'close'
target_word_idx = mn_r100['item'].index(target_word)
bias_name = 'bias_s'

df_dict = {}
df_dict["target_act"] = mn_r100[layer][target_word_idx, :]
df_dict["bias"] = [w.numpy() for w in model.weights if w.name.startswith(bias_name)][0]
df_time_invar = pd.DataFrame.from_dict(df_dict)
df_time_invar["unit"] = df_time_invar.index
df_time_invar["word"] = target_word
df_time_invar

In [ ]:
SEM_NAME_MAP = {
    "input_hps_hs": "PS",
    "input_css_cs": "CS",
    "input_sem": "SS",
    "input_hos_hs": "OS",
    "input_sem": "input",
    "sem": "act",
}
PHO_NAME_MAP = {
    "input_hsp_hp": "SP",
    "input_cpp_cp": "CP",
    "input_pho_pp": "PP",
    "input_hop_hp": "OP",
    "input_pho": "input",
    "pho": "act",
}

name_map = SEM_NAME_MAP

df_time_varying = pd.DataFrame()

for i, model_output_name in enumerate(name_map.keys()):
    this_output_df = pd.DataFrame()
    for t in range(13):
        df_dict = {}
        name = name_map[model_output_name]
        df_dict[name] = y_pred[model_output_name][t, target_word_idx, :]
        this_step_df = pd.DataFrame.from_dict(df_dict)
        this_step_df["timetick"] = t
        this_step_df["unit"] = this_step_df.index
        this_output_df = pd.concat([this_output_df, this_step_df], ignore_index=True)

    if i == 0:
        df_time_varying = this_output_df
    else:
        df_time_varying = pd.merge(
            df_time_varying, this_output_df, on=["timetick", "unit"]
        )

df_time_varying

In [ ]:
# Merge and export
df = df_time_varying.merge(df_time_invar, on="unit", how="left")
df["unit_acc"] = abs(df.target_act - df.act) < 0.5
df = df[
    ["word", "unit", "unit_acc", "timetick", "target_act", "bias"]
    + list(name_map.values())
]

# Restructure
melt_value_vars = ["bias"] + list(name_map.values())
df = df.melt(
    id_vars=["word", "unit", "timetick", "target_act", "unit_acc"],
    value_vars=melt_value_vars,
)

In [ ]:
all_on_nodes = df.loc[df.target_act == 1, "unit"].unique()
all_off_nodes = df.loc[df.target_act == 0, "unit"].unique()
print(f"On: {all_on_nodes} \nOff: {np.random.choice(all_off_nodes, 10)}")

In [ ]:
import altair as alt
sel_node = all_on_nodes
node_df = df.loc[df.unit.isin(sel_node)]
plot_df = node_df.loc[~node_df.variable.isin(['act', 'input', 'SS', 'PP'])]

alt.Chart(plot_df).mark_line().encode(
    y='mean(value):Q', 
    x='timetick:Q', 
    color='variable:N'
    ).properties(title=f"word: {target_word} at nodes: {sel_node}")

## EoT acc

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
import data_wrangling
def min_cosine_distance_idx(all_reps, pred):
    """return the index of word that has min cosine distance"""
    all_cosine_dist = [cosine(pred, rep) for rep in all_reps]
    return np.argmin(all_cosine_dist) 

def cosine_accuracy(all_reps, pred, target):
    """Check whether the prediction is the min cosine distance word"""
    target_idx = min_cosine_distance_idx(all_reps, target)
    pred_idx = min_cosine_distance_idx(all_reps, pred)
    return target_idx == pred_idx

def all_cosine_accuracy(all_reps, preds, targets):
    return np.mean([cosine_accuracy(all_reps, pred, target) for pred, target in zip(preds, targets)])

def binary_accuracy(pred, target):
    """Calculate correct side accuracy"""
    d = abs(pred - target)
    max_d = np.max(d, axis = 1)
    # print(max_d)
    return np.mean(max_d < 0.5)

def get_all_acc(y_pred):
    """Calculate accuracy of all outputs"""
    mn_train = data_wrangling.load_testset('mn_r100')
    sem_acc = all_cosine_accuracy(all_reps=mn_train['sem'], preds=y_pred['sem'][-1].numpy(), targets=mn_train['sem'])
    pho_acc = binary_accuracy(mn_r100['pho'], y_pred['pho'][-1, :, :].numpy())
    return {
        "pho": pho_acc,
        "sem": sem_acc
    }

def get_task_acc(task):
    """Get task acc"""
    model.set_active_task(task)
    input_name = modeling.IN_OUT[task][0]
    y_pred = model([mn_r100[input_name]] * cfg.n_timesteps)
    return get_all_acc(y_pred)

In [ ]:
get_task_acc('triangle')